In [59]:
# Load necessary libraries
if (!requireNamespace("BiocManager", quietly = TRUE)) {
  install.packages("BiocManager")
}
BiocManager::install(c("minfi", "limma"))
install.packages("caret")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.0 (2023-04-21)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'minfi' 'limma'”
Installation paths not writeable, unable to update packages
  path: /u/local/apps/R/4.3.0/gcc10.2.0_intel2022.1.1/lib64/R/library
  packages:
    BiocGenerics, BiocParallel, Biostrings, bitops, boot, bslib, cachem, class,
    cli, cluster, codetools, colorspace, cpp11, crayon, curl, data.table,
    deldir, digest, dqrng, evaluate, fansi, farver, fastDummies, fastmap,
    fitdistrplus, foreign, fs, future, future.apply, GenomeInfoDb,
    GenomeInfoDbData, GenomicRanges, ggplot2, globals, glue, gtable, highr,
    htmltools, httpuv, igraph, IRanges, jsonlite, KernSmooth, lattice,
    l

In [3]:
library(minfi)
library(limma)
library(caret) # For data splitting
library(survival)
library(dplyr)
library(data.table)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loa

In [2]:
transformed_methylation_data <- read.csv("/u/home/c/ctang04/Liver Project/data/transformed_log1k_methylation_data.csv", row.names = 1, check.names=FALSE)
# Load phenotype data
phenotype_matrix <- read.csv("/u/home/c/ctang04/Liver Project/data/patients.info.for.liver.study_with.seq.data_2024.07.21.csv", check.names = FALSE)

In [1]:
# Read the transformed methylation data
transformed_methylation_data <- readRDS("/u/home/c/ctang04/Liver Project/data/transformed_methyl_data_frame.rds")

# Read the phenotype matrix
phenotype_matrix <- readRDS("/u/home/c/ctang04/Liver Project/data/phenotype_matrix.rds")

dim(transformed_methylation_data)
dim(phenotype_matrix)

[1] 144965   5610

[1] 647  69

In [2]:
# Extract unique sample aliases from phenotype data
phenotype_sample_names <- phenotype_matrix$plasma_alias
split_pheno_names <- unlist(strsplit(as.character(phenotype_sample_names), "; "))
unique_sample_aliases <- unique(split_pheno_names)

# Match sample names between methylation and phenotype data
sample_names <- colnames(transformed_methylation_data)
matches <- sapply(sample_names, function(sample) any(grepl(sample, unique_sample_aliases)))
matched_sample_names <- sample_names[matches]

# Filter methylation data to include only matched samples
filtered_methyl_data <- transformed_methylation_data[, matched_sample_names, drop = FALSE]

# Split phenotype data to match the filtered methylation data
split_phenotype_data <- phenotype_matrix[rep(seq_len(nrow(phenotype_matrix)), sapply(phenotype_matrix$plasma_alias, function(x) length(strsplit(x, "; ")[[1]]))), ]
split_phenotype_data$plasma_alias <- unlist(strsplit(as.character(phenotype_matrix$plasma_alias), "; "))
split_phenotype_data <- split_phenotype_data[!is.na(split_phenotype_data$plasma_alias), ]

# Match common samples between methylation and phenotype data
common_samples <- intersect(colnames(filtered_methyl_data), split_phenotype_data$plasma_alias)
filtered_pheno_data <- split_phenotype_data[split_phenotype_data$plasma_alias %in% common_samples, ]
filtered_pheno_data <- filtered_pheno_data[match(colnames(filtered_methyl_data), filtered_pheno_data$plasma_alias), ]

# Combine methylation and phenotype data
transposed_methyl_data <- as.data.frame(t(filtered_methyl_data))
print(dim(transposed_methyl_data))
combined_data <- cbind(transposed_methyl_data, filtered_pheno_data)
print(dim(combined_data))

# Ensure no NA values in 'alt_U_L'
combined_data_clean <- combined_data[!is.na(combined_data$alt_U_L), ]
print(dim(combined_data_clean))

[1]    578 144965
[1]    578 145034
[1]    563 145034


In [3]:
combined_data_clean_aliases <- combined_data_clean[,"plasma_alias"]
rownames(combined_data_clean) <- combined_data_clean_aliases

In [8]:
write.csv(combined_data_clean, "/u/home/c/ctang04/Liver Project/data/alt_samples_withdup.csv")

In [4]:
# Remove duplicates in place by selecting the first sample for each donor
rownames_combined <- rownames(combined_data_clean)

combined_data_clean <- combined_data_clean %>%
  mutate(rowname = rownames_combined) %>%
  group_by(donor_id) %>%
  slice(1) %>%
  ungroup()

print(combined_data_clean$plasma_alias)

ERROR: Error in combined_data_clean %>% mutate(rowname = rownames_combined) %>% : could not find function "%>%"


In [ ]:
combined_data_clean_aliases_nodup <- combined_data_clean$plasma_alias
print(combined_data_clean_aliases_nodup)

In [11]:
#write.csv(combined_data_clean, "/u/home/c/ctang04/Liver Project/data/alt_samples_nodup.csv")

In [ ]:
as.data.frame(combined_data_clean)
rownames(combined_data_clean) <- combined_data_clean$plasma_alias

In [ ]:
# Check if 'plasma_alias' is a column in the dataframe
"plasma_alias" %in% colnames(combined_data_clean)
"alt_U_L" %in% colnames(combined_data_clean)

In [ ]:
max_alt_U_L <- max(combined_data_clean$alt_U_L, na.rm = TRUE)
min_alt_U_L <- min(combined_data_clean$alt_U_L, na.rm = TRUE)

# Print largest and smallest value
cat("The largest value in alt_U_L is:", max_alt_U_L, "\n")
cat("The smallest value in alt_U_L is:", min_alt_U_L, "\n")

# Log Transformation

In [ ]:
# Ensure all values in alt_U_L are positive by adding a small constant (if necessary)
min_alt <- min(combined_data_clean$alt_U_L, na.rm = TRUE)
if (min_alt <= 0) {
  combined_data_clean$alt_U_L <- combined_data_clean$alt_U_L + (1 - min_alt)
}

# Apply log transformation to alt_U_L
combined_data_clean$log_alt_U_L <- log(combined_data_clean$alt_U_L)

# View the first few rows to verify the transformation
combined_data_clean <- as.data.frame(combined_data_clean)
rownames(combined_data_clean) <- combined_data_clean$plasma_alias
head(combined_data_clean)
# Convert to data.table and ensure all columns are numeric where appropriate
combined_data_clean_dt <- as.data.table(combined_data_clean)

# Removing Columns

In [ ]:
columns_to_remove <- colnames(filtered_pheno_data)

# Remove columns in `combined_data_clean` that are present in `columns_to_remove`
combined_data_clean_filtered <- combined_data_clean[, !colnames(combined_data_clean) %in% columns_to_remove]

# Verify the dimensions of the cleaned dataset
print(dim(combined_data_clean_filtered))

In [ ]:
non_numeric_cols <- sapply(combined_data_clean, function(x) !is.numeric(x) && !is.integer(x))
print(names(combined_data_clean_dt)[non_numeric_cols])

In [ ]:
dim(combined_data)
dim(combined_data_clean)

# Splitting Data

In [15]:
# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Create k-fold cross-validation indices
folds <- createFolds(combined_data_clean$log_alt_U_L, k = k, list = TRUE, returnTrain = TRUE)

# Perform k-fold cross-validation manually and save each fold's data
for (i in 1:k) {
  # Get the training and testing indices
    train_indices <- folds[[i]]
    test_indices <- setdiff(1:nrow(combined_data_clean), train_indices)
  
    # Subset the training and testing data
    training_set <- combined_data_clean[train_indices, ]
    testing_set <- combined_data_clean[test_indices, ]

  # Save the data as .rds
    saveRDS(training_set, file = paste0("/u/home/c/ctang04/Liver Project/data/alt_training_set_fold_", i, ".rds"))
    saveRDS(testing_set, file = paste0("/u/home/c/ctang04/Liver Project/data/alt_testing_set_fold_", i, ".rds"))
}

In [ ]:
# Read .rds files
k<-5
training_sets <- lapply(1:k, function(i) readRDS(paste0("/u/home/c/ctang04/Liver Project/data/alt_training_set_fold_", i, ".rds")))
testing_sets <- lapply(1:k, function(i) readRDS(paste0("/u/home/c/ctang04/Liver Project/data/alt_testing_set_fold_", i, ".rds")))

# 5 Fold Cross Validation Method

# Predicting Values

In [ ]:
library(caret)
library(limma)
library(ggplot2)

# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Initialize lists to store results
all_predictions <- list()
all_performances <- vector()
all_top_markers <- list()

# Perform k-fold cross-validation manually
for (i in 1:k) {
    # Use the preloaded training and testing sets
    training_set <- training_sets[[i]]
    testing_set <- testing_sets[[i]]

    # Ensure the column names are correct for column removal
    columns_to_remove <- colnames(filtered_pheno_data)
    columns_to_remove <- c(columns_to_remove, "rowname") # Adjust if needed

    # Remove specified columns from the training data
    methylation_training_data_filtered <- training_set[, !colnames(training_set) %in% columns_to_remove]

    # Create the design matrix
    design_train <- model.matrix(~ log_alt_U_L, data = training_set)

    # Remove 'log_alt_U_L' column from methylation data
    methylation_data_no_log_alt <- methylation_training_data_filtered[, !colnames(methylation_training_data_filtered) %in% "log_alt_U_L"]

    # Check dimensions
    print(dim(design_train))  # Dimensions of design matrix
    print(dim(methylation_data_no_log_alt))  # Dimensions of filtered data

    # Fit the linear model
    fit <- lmFit(t(methylation_data_no_log_alt), design_train)
    fit <- eBayes(fit)

    # Get top 100 significant markers
    top_100_results <- topTable(fit, coef = 2, number = 100)
    #print(head(top_100_results))
    top_100_markers <- rownames(top_100_results)
    # Get top markers with logFC > 1.2 (log scale) and lowest p-value
    top_results <- topTable(fit, coef = 2, number = Inf)

    # Exponentiate the logFC column (assuming base 2)
    top_results$FoldChange <- 2^top_results$logFC

    #print(head(top_results))

    top_results_filtered <- top_results[abs(top_results$FoldChange) > 1.2, ]
    print(dim(top_results_filtered))
    top_results_sorted <- top_results_filtered[order(top_results_filtered$P.Value), ]
    print(dim(top_results_sorted))
    top_100_markers_FC <- head(rownames(top_results_sorted), 100)
    # Store the top 100 markers for the current fold
    all_top_markers[[i]] <- top_100_markers
    # Export top 100 markers for the current fold to a CSV file
    write.csv(top_100_markers, file = paste0("ALT_Linear_top_100_markers_fold_", i, ".csv"), row.names = FALSE)

    # Filter training and testing data to include only the top 100 markers
    training_set_filtered <- training_set[, c("log_alt_U_L", top_100_markers)]
    testing_set_filtered <- testing_set[, c("log_alt_U_L", top_100_markers)]

    # Fit the linear model using the filtered training data
    model <- lm(log_alt_U_L ~ ., data = training_set_filtered)

    # Predict on the filtered testing set
    log_predictions <- predict(model, newdata = testing_set_filtered)
    predictions <- exp(log_predictions)

    # Store the predictions
    all_predictions[[i]] <- data.frame(
        PlasmaAlias = testing_set$plasma_alias,
        Actual = testing_set$alt_U_L,
        Predicted = predictions
    )

    # Calculate and store the performance metrics for the current fold
    performance <- postResample(predictions, testing_set$log_alt_U_L)
    all_performances <- rbind(all_performances, performance)

    # Plot the actual vs predicted values
    plot_data <- data.frame(
        Actual = testing_set$alt_U_L,
        Predicted = predictions
    )

    # Calculate Pearson Correlation Coefficient R and R2
    r_value <- cor(plot_data$Actual, plot_data$Predicted)
    r_squared <- r_value^2
    pearson_corr <- cor(log(plot_data$Actual), log(plot_data$Predicted), method = "pearson")
    # Create the plot for log-transformed values with y = x line and Pearson correlation
    p <- ggplot(plot_data, aes(x = log(Actual), y = log(Predicted))) +
        geom_point(color = "blue") +
        #geom_smooth(method = "lm", color = "red", se = FALSE) + # Add a linear regression line
        geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "black") + # Add the y = x line
        labs(title = paste("Fold", i, "Log-Transformed Actual vs Predicted"),
             subtitle = paste("R =", round(r_value, 2), "| R² =", round(r_squared, 2), "| Pearson =", round(pearson_corr, 2)),
             x = "Log Actual Values",
             y = "Log Predicted Values") +
        #coord_cartesian(xlim = c(0, log(150)), ylim = c(0, log(150))) + # Set limits for x and y axes on log scale
        theme_minimal()
    print(p)
}
# Calculate average performance across all folds
print(all_performances)
mean_performance <- colMeans(all_performances)
print(mean_performance)

# Print the top 100 markers for each fold
# for (i in 1:k) {
#   cat(paste("Top 100 markers for fold", i, ":\n"))
#   print(all_top_markers[[i]])
# }

# Find shared top markers across all folds after the loop
shared_markers <- Reduce(intersect, all_top_markers)

# Print the shared markers to a CSV file
write.csv(shared_markers, file = "ALT_shared_top_100_markers.csv", row.names = FALSE)

# Print the shared markers
cat("Shared markers across all folds:\n")
print(shared_markers)

In [ ]:
library(ggplot2)

# Combine all predictions from the k folds
combined_predictions <- do.call(rbind, all_predictions)

# Calculate Pearson Correlation Coefficient R and R² for combined data
r_value_combined <- cor(combined_predictions$Actual, combined_predictions$Predicted)
r_squared_combined <- r_value_combined^2
pearson_corr_combined <- cor(log(combined_predictions$Actual), log(combined_predictions$Predicted), method = "pearson")

# Create the scatter plot for log-transformed values with y = x line
overall_plot <- ggplot(combined_predictions, aes(x = log(Actual), y = log(Predicted))) +
  geom_point(color = "blue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "black") + # Add y = x line
  labs(title = "Overall Log-Transformed Actual vs Predicted",
       subtitle = paste("R =", round(r_value_combined, 2), "| R² =", round(r_squared_combined, 2), "| Pearson =", round(pearson_corr_combined, 2)),
       x = "Log Actual Values",
       y = "Log Predicted Values") +
  theme_minimal()

# Print the overall scatter plot
print(overall_plot)


# Linear Classification

In [ ]:
library(caret)
library(limma)
library(pROC)

# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Initialize lists to store results
all_conf_matrices <- list()
all_top_markers <- list()
all_auc <- list()
all_correlations <- numeric(k)

# Define classification thresholds
classify <- function(value) {
  if (value <= 40) {
    return(0)
  } else {
    return(1)
  }
}

# Perform k-fold cross-validation manually
for (i in 1:k) {
    # Use the preloaded training and testing sets
    training_set <- training_sets[[i]]
    testing_set <- testing_sets[[i]]

    # Ensure the column names are correct for column removal
    columns_to_remove <- colnames(filtered_pheno_data)
    columns_to_remove <- c(columns_to_remove, "rowname") # Adjust if needed

    # Remove specified columns from the training data
    methylation_training_data_filtered <- training_set[, !colnames(training_set) %in% columns_to_remove]

    # Classify the training data based on the cutoff
    training_set$class <- sapply(training_set$alt_U_L, classify)

    # Create the design matrix using the classified training data
    design_train <- model.matrix(~ class, data = training_set)

    # Remove 'log_alt_U_L' column from methylation data
    methylation_data_no_log_alt <- methylation_training_data_filtered[, !colnames(methylation_training_data_filtered) %in% "log_alt_U_L"]

    # Check dimensions
    print(dim(design_train))
    print(dim(methylation_data_no_log_alt))

    # Fit the linear model
    fit <- lmFit(t(methylation_data_no_log_alt), design_train)
    fit <- eBayes(fit)

    # Get top 100 significant markers
    top_100_results <- topTable(fit, coef = 2, number = 100)
    top_100_markers <- rownames(top_100_results)

    # Store the top 100 markers for the current fold
    all_top_markers[[i]] <- top_100_markers
    # Export top 100 markers for the current fold to a CSV file
    write.csv(top_100_markers, file = paste0("ALT_Linear_top_100_markers_fold_", i, ".csv"), row.names = FALSE)

    # Filter training and testing data to include only the top 100 markers
    training_set_filtered <- training_set[, c("log_alt_U_L", top_100_markers)]
    testing_set_filtered <- testing_set[, c(top_100_markers)]

    # Fit the linear model using the filtered training data
    model <- lm(log_alt_U_L ~ ., data = training_set_filtered)

    # Predict on the filtered testing set
    log_predictions <- predict(model, newdata = testing_set_filtered)
    predictions <- exp(log_predictions)  # Exponentiate the predictions

    # Exponentiate the actual values for comparison
    actual_values <- testing_set$alt_U_L

    # Print actual and predicted values before classification
    print(data.frame(
        PlasmaAlias = testing_set$plasma_alias,
        Actual = actual_values,
        Predicted = predictions
    ))

    # Classify predictions and actual values
    classified_predictions <- sapply(predictions, classify)
    classified_actuals <- sapply(actual_values, classify)

    # Convert predictions and actual values to factors
    classified_predictions <- factor(classified_predictions, levels = 0:1)
    classified_actuals <- factor(classified_actuals, levels = 0:1)

    # Create a confusion matrix
    conf_matrix <- confusionMatrix(classified_predictions, classified_actuals)
    all_conf_matrices[[i]] <- conf_matrix

    # Calculate Pearson's correlation coefficient
    correlation <- cor(actual_values, predictions)
    all_correlations[i] <- correlation

    # Calculate AUC using actual class labels and prediction probabilities
    roc_curve <- roc(classified_actuals, as.numeric(classified_predictions))
    auc_value <- auc(roc_curve)
    all_auc[i] <- auc_value

    # Print confusion matrix, Pearson's correlation, and AUC for the current fold
    cat(paste("Fold", i, "Confusion Matrix:\n"))
    print(conf_matrix)
    cat(paste("Fold", i, "Pearson's Correlation Coefficient:", correlation, "\n"))
    cat(paste("Fold", i, "AUC:", auc_value, "\n"))
}

# Print confusion matrices, correlation coefficients, and AUC for each fold
for (i in 1:k) {
  cat(paste("Confusion Matrix for fold", i, ":\n"))
  print(all_conf_matrices[[i]])
  cat(paste("Fold", i, "Pearson's Correlation Coefficient:", all_correlations[i], "\n"))
  cat(paste("Fold", i, "AUC:", all_auc[i], "\n"))
}
# Calculate and print the mean AUC across all folds
mean_auc <- mean(unlist(all_auc))
cat(paste("Mean AUC across all folds:", mean_auc, "\n"))


In [5]:
library(caret)
library(limma)
library(pROC)

# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Initialize lists to store results
all_results <- list()  # A list to hold all metrics
all_top_markers <- list()
all_auc <- numeric(k)
all_correlations <- numeric(k)

# Define classification thresholds
classify <- function(value) {
  if (value <= 40) {
    return(0)
  } else {
    return(1)
  }
}

# Perform k-fold cross-validation manually
for (i in 1:k) {
    # Use the preloaded training and testing sets
    training_set <- training_sets[[i]]
    testing_set <- testing_sets[[i]]

    # Ensure the column names are correct for column removal
    columns_to_remove <- colnames(filtered_pheno_data)
    columns_to_remove <- c(columns_to_remove, "rowname") # Adjust if needed

    # Remove specified columns from the training data
    methylation_training_data_filtered <- training_set[, !colnames(training_set) %in% columns_to_remove]

    # Classify the training data based on the cutoff
    training_set$class <- sapply(training_set$alt_U_L, classify)

    # Create the design matrix using the classified training data
    design_train <- model.matrix(~ class, data = training_set)

    # Remove 'log_alt_U_L' column from methylation data
    methylation_data_no_log_alt <- methylation_training_data_filtered[, !colnames(methylation_training_data_filtered) %in% "log_alt_U_L"]

    # Fit the linear model
    fit <- lmFit(t(methylation_data_no_log_alt), design_train)
    fit <- eBayes(fit)

    # Get top 100 significant markers
    top_100_results <- topTable(fit, coef = 2, number = 100)
    top_100_markers <- rownames(top_100_results)

    # Store the top 100 markers for the current fold
    all_top_markers[[i]] <- top_100_markers
    write.csv(top_100_markers, file = paste0("ALT_Linear_top_100_markers_fold_", i, ".csv"), row.names = FALSE)

    # Filter training and testing data to include only the top 100 markers
    training_set_filtered <- training_set[, c("log_alt_U_L", top_100_markers)]
    testing_set_filtered <- testing_set[, c(top_100_markers)]

    # Fit the linear model using the filtered training data
    model <- lm(log_alt_U_L ~ ., data = training_set_filtered)

    # Predict on the filtered testing set
    log_predictions <- predict(model, newdata = testing_set_filtered)
    predictions <- exp(log_predictions)

    actual_values <- testing_set$alt_U_L

    # Classify predictions and actual values
    classified_predictions <- sapply(predictions, classify)
    classified_actuals <- sapply(actual_values, classify)

    # Convert predictions and actual values to factors
    classified_predictions <- factor(classified_predictions, levels = 0:1)
    classified_actuals <- factor(classified_actuals, levels = 0:1)

    # Create a confusion matrix
    conf_matrix <- confusionMatrix(classified_predictions, classified_actuals)

    # Calculate Pearson's correlation coefficient
    correlation <- cor(actual_values, predictions)
    all_correlations[i] <- correlation

    # Calculate AUC using actual class labels and prediction probabilities
    roc_curve <- roc(classified_actuals, as.numeric(classified_predictions))
    auc_value <- auc(roc_curve)
    all_auc[i] <- auc_value

    # Store all the results for the current fold in a list
    all_results[[i]] <- list(
        Fold = i,
        ConfusionMatrix = conf_matrix,
        Correlation = correlation,
        AUC = auc_value
    )
}

# Display all the results for each fold in a grouped manner
for (i in 1:k) {
  cat(paste("\nFold", all_results[[i]]$Fold, "Results:"))
  cat("\nConfusion Matrix:\n")
  print(all_results[[i]]$ConfusionMatrix)
  cat(paste("\nPearson's Correlation Coefficient:", all_results[[i]]$Correlation))
  cat(paste("\nAUC:", all_results[[i]]$AUC, "\n"))
}

# Calculate and print the mean AUC across all folds
mean_auc <- mean(unlist(all_auc))
cat(paste("\nMean AUC across all folds:", mean_auc, "\n"))

# Calculate and print the mean correlation across all folds
mean_correlation <- mean(all_correlations)
cat(paste("\nMean Pearson's Correlation Coefficient across all folds:", mean_correlation, "\n"))


Loading required package: ggplot2

Loading required package: lattice

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




ERROR: Error in eval(expr, envir, enclos): object 'training_sets' not found


# Random Forest

In [ ]:
library(caret)
library(limma)
library(randomForest)
library(pROC)

# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Initialize lists to store results
all_conf_matrices <- list()
all_top_markers <- list()
all_auc <- numeric(k)

# Define classification thresholds
classify <- function(value) {
  if (value <= 40) {
    return(1)
  } else {
    return(2)
  }
}

# Perform k-fold cross-validation manually
for (i in 1:k) {
    # Use the preloaded training and testing sets
    training_set <- training_sets[[i]]
    testing_set <- testing_sets[[i]]

    # Ensure the column names are correct for column removal
    columns_to_remove <- colnames(filtered_pheno_data)
    columns_to_remove <- c(columns_to_remove, "rowname") # Adjust if needed

    # Remove specified columns from the training data
    methylation_training_data_filtered <- training_set[, !colnames(training_set) %in% columns_to_remove]

    # Apply the classify function to log_alt_U_L to create a categorical target variable
    training_set$log_alt_U_L_class <- factor(sapply(training_set$log_alt_U_L, classify), levels = 1:2)

    # Create the design matrix for marker selection
    design_train <- model.matrix(~ log_alt_U_L, data = training_set)

    # Remove 'log_alt_U_L' column from methylation data
    methylation_data_no_log_alt <- methylation_training_data_filtered[, !colnames(methylation_training_data_filtered) %in% "log_alt_U_L"]

    # Fit the linear model
    fit <- lmFit(t(methylation_data_no_log_alt), design_train)
    fit <- eBayes(fit)

    # Get top 100 significant markers
    top_100_results <- topTable(fit, coef = 2, number = 100)
    top_100_markers <- rownames(top_100_results)

    # Store the top 100 markers for the current fold
    all_top_markers[[i]] <- top_100_markers

    # Filter training and testing data to include only the top 100 markers
    training_set_filtered <- training_set[, c("log_alt_U_L", top_100_markers)]
    testing_set_filtered <- testing_set[, top_100_markers]

    # Fit the Random Forest model using the filtered training data
    rf_model <- randomForest(log_alt_U_L ~ ., data = training_set_filtered, importance = TRUE)

    # Predict continuous values on the filtered testing set
    log_predictions <- predict(rf_model, newdata = testing_set_filtered)
    predictions <- exp(log_predictions)  # Exponentiate the predictions

    # Exponentiate the actual values for comparison
    actual_values <- testing_set$alt_U_L

    # Print actual and predicted values before classification
    print(data.frame(
        PlasmaAlias = testing_set$plasma_alias,
        Actual = actual_values,
        Predicted = predictions
    ))

    # Classify the continuous predictions into class labels
    classified_predictions <- sapply(predictions, classify)
    classified_actuals <- sapply(actual_values, classify)

    # Convert predictions and actual values to factors
    classified_predictions <- factor(classified_predictions, levels = 1:2)
    classified_actuals <- factor(classified_actuals, levels = 1:2)

    # Create a confusion matrix
    conf_matrix <- confusionMatrix(classified_predictions, classified_actuals)
    all_conf_matrices[[i]] <- conf_matrix

    # Calculate ROC curve and AUC
    roc_curve <- roc(classified_actuals, as.numeric(classified_predictions))
    auc_value <- auc(roc_curve)
    all_auc[i] <- auc_value

    # Print confusion matrix and AUC for the current fold
    cat(paste("Fold", i, "Confusion Matrix:\n"))
    print(conf_matrix)
    cat(paste("Fold", i, "AUC:", auc_value, "\n"))
}

# Print confusion matrices and AUCs for each fold)
for (i in 1:k) {
  cat(paste("Confusion Matrix for fold", i, ":\n"))
  print(all_conf_matrices[[i]])
  cat(paste("Fold", i, "AUC:", all_auc[i], "\n"))
}
# Calculate and print the mean AUC across all folds
mean_auc <- mean(unlist(all_auc))
cat(paste("Mean AUC across all folds:", mean_auc, "\n"))


In [ ]:
library(caret)
library(limma)
library(randomForest)
library(pROC)

# Set seed for reproducibility
set.seed(123)

# Number of folds
k <- 5

# Initialize lists to store results
all_conf_matrices <- list()
all_top_markers <- list()
all_auc <- numeric(k)

# Define classification thresholds
classify <- function(value) {
  if (value <= log(40)) {
    return(1)
  } else {
    return(2)
  }
}

# Perform k-fold cross-validation manually
for (i in 1:k) {
    # Use the preloaded training and testing sets
    training_set <- training_sets[[i]]
    testing_set <- testing_sets[[i]]

    # Ensure the column names are correct for column removal
    columns_to_remove <- colnames(filtered_pheno_data)
    columns_to_remove <- c(columns_to_remove, "rowname") # Adjust if needed

    # Remove specified columns from the training data
    methylation_training_data_filtered <- training_set[, !colnames(training_set) %in% columns_to_remove]

    # Apply the classify function to log_alt_U_L to create a categorical target variable
    training_set$log_alt_U_L_class <- factor(sapply(training_set$log_alt_U_L, classify), levels = 1:2)

    # Create the design matrix for marker selection
    design_train <- model.matrix(~ log_alt_U_L, data = training_set)

    # Remove 'log_alt_U_L' column from methylation data
    methylation_data_no_log_alt <- methylation_training_data_filtered[, !colnames(methylation_training_data_filtered) %in% "log_alt_U_L"]

    # Fit the linear model
    fit <- lmFit(t(methylation_data_no_log_alt), design_train)
    fit <- eBayes(fit)

    # Get top 100 significant markers
    top_100_results <- topTable(fit, coef = 2, number = 100)
    top_100_markers <- rownames(top_100_results)

    # Store the top 100 markers for the current fold
    all_top_markers[[i]] <- top_100_markers

    # Filter training and testing data to include only the top 100 markers
    training_set_filtered <- training_set[, c("log_alt_U_L_class", top_100_markers)]
    testing_set_filtered <- testing_set[, top_100_markers]

    # Fit the Random Forest model using the filtered training data to predict the class
    rf_model <- randomForest(log_alt_U_L_class ~ ., data = training_set_filtered, importance = TRUE)

    # Predict the class directly on the filtered testing set
    class_predictions <- predict(rf_model, newdata = testing_set_filtered)

    # Actual class values based on `alt_U_L`
    actual_values <- factor(sapply(testing_set$alt_U_L, classify), levels = 1:2)

    # Print actual and predicted class values
    print(data.frame(
        PlasmaAlias = testing_set$plasma_alias,
        Actual = actual_values,
        Predicted = class_predictions
    ))

    # Create a confusion matrix
    conf_matrix <- confusionMatrix(class_predictions, actual_values)
    all_conf_matrices[[i]] <- conf_matrix

    # Calculate ROC curve and AUC
    roc_curve <- roc(actual_values, as.numeric(class_predictions))
    auc_value <- auc(roc_curve)
    all_auc[i] <- auc_value

    # Print confusion matrix and AUC for the current fold
    cat(paste("Fold", i, "Confusion Matrix:\n"))
    print(conf_matrix)
    cat(paste("Fold", i, "AUC:", auc_value, "\n"))
}

# Print confusion matrices and AUCs for each fold
for (i in 1:k) {
  cat(paste("Confusion Matrix for fold", i, ":\n"))
  print(all_conf_matrices[[i]])
  cat(paste("Fold", i, "AUC:", all_auc[i], "\n"))
}

# Calculate and print the mean AUC across all folds
mean_auc <- mean(unlist(all_auc))
cat(paste("Mean AUC across all folds:", mean_auc, "\n"))
